## Import packages

In [3]:
from big_phoney import BigPhoney
import lyricsgenius as genius
import random
import nlpaug.augmenter.word as naw
import re
import numpy as np

#----------------For future live text generation-----------------
#import gpt_2_simple as gpt2

# model_name = "124M"
# model is saved into current directory under /models/124M/
# gpt2.download_gpt2(model_name=model_name) #need to run only once. comment out once done.

# Load models
#sess = gpt2.start_tf_sess()
#gpt2.load_gpt2(sess, run_name='run1')

#----------------------------------------------------------------

## Functions 

In [6]:
def get_target_lyrics(artist, title, api_key):
    
    api = genius.Genius(api_key)
    api.remove_section_headers = True # Removes section headers like "Verse" and "Intro"
    
    #get song lyrics
    song = api.search_song(title, artist)
    
    #clean up the target lyrics
    target = song.lyrics
    target = target.replace("\u2005"," ")
    target = target.replace("\\"," ")
    target = target.replace("\\n"," ")
    target = target.replace("("," ")
    target = target.replace(")"," ")
    target = target.replace("\n\n","\n")
    target = target.replace("\n\n\n","\n")
    target = target.replace("2x","")
    #create list of lines
    target_lyrics = target.split("\n")
    print("Target lyrics:")
    print(target_lyrics)
    return target_lyrics

In [7]:
get_target_lyrics('Idina Menzel','Let it go')

Searching for "Let it go" by Idina Menzel...
Done.
Target lyrics:
['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, heaven knows I've tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on', 'The cold never bothered me anyway', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', 'Let it go  go, go, go go, go go, go go, go, go, go go ', 'Let it go', 'Let it go', 'Let it go', "It's funny how some distance makes everything seem small", "And the fears that once controlled me can't get to me at all"

['The snow glows white on the mountain tonight',
 'Not a footprint to be seen',
 'A kingdom of isolation',
 "And it looks like I'm the queen",
 'The wind is howling like this swirling storm inside',
 "Couldn't keep it in, heaven knows I've tried",
 "Don't let them in, don't let them see",
 'Be the good girl you always have to be',
 "Conceal, don't feel, don't let them know",
 'Well, now they know',
 'Let it go, let it go',
 "Can't hold it back anymore",
 'Let it go, let it go',
 'Turn away and slam the door',
 "I don't care what they're going to say",
 'Let the storm rage on',
 'The cold never bothered me anyway',
 'Let it go, let it go',
 "Can't hold it back anymore",
 'Let it go, let it go',
 'Turn away and slam the door',
 'Let it go  go, go, go go, go go, go go, go, go, go go ',
 'Let it go',
 'Let it go',
 'Let it go',
 "It's funny how some distance makes everything seem small",
 "And the fears that once controlled me can't get to me at all",
 "It's time to see what I can do",
 'T

In [61]:
#----------------For future live text generation-----------------
#def generate_lyrics(): 
#    
#    gen_lyrics = gpt2.generate(sess, 
#                     length=250,
#                     temperature=0.9,
#                     include_prefix=False,
#                     #prefix="Deep in the night",
#                     nsamples=1,
#                     batch_size=1,
#                     return_as_list=True)[0]
#    
#    gen_lyrics = gen_lyrics.split("\n")
#    
#    return gen_lyrics
#----------------------------------------------------------------

In [127]:
def lyrics_to_list(file_name):
    with open(file_name, 'r') as file:
        data = file.read().split("====================")
        random_song = random.choice(data)
        target = random_song
        
        # clean generatedd lyrics
        target = target.replace("\u2005"," ")
        target = target.replace("\\"," ")
        target = target.replace("\\n"," ")
        target = target.replace("("," ")
        target = target.replace(")"," ")
        target = target.replace("\n\n","\n")
        target = target.replace("\n\n\n","\n")
        target = target.replace("2x","")
        gen_lyrics = target.split("\n")
        gen_lyrics = gen_lyrics[1:-1]
        print("Gen lyrics:")
        print(gen_lyrics)
    return gen_lyrics

#### The magical lyrics fitting function

This function takes lyrics generated by a fine-tuned GPT-2 model, and fits them to the syllabic structure of target lyrics (matches the number of syllables per line). 

In [129]:
def fit_lyrics(gen_lyrics, target_lyrics):
    
    print(gen_lyrics)
    
    aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
    print('aug')
    
    phoney = BigPhoney()
    print('initialized phoney')
    
    # Counts the number of syllables in each line
    def count_syls(text):
    
        schema = []
    
        for line in text:
            syls = phoney.count_syllables(line)
            schema.append(syls)
            #print(syls,line)
        
        return schema
    
    # Remove characters and contractions from the line. This will make it easier to
    # augment lines that need augmentation. 
    def decontracted(phrase):
        # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        phrase = re.sub('[!@#$?]', '', phrase)
        return phrase
    
    # Count the number of syllables in the generated and target texts.
    gen_schema = count_syls(gen_lyrics)
    target_schema = count_syls(target_lyrics)
#    print(target_schema)

    # make generated lyrics same length as target lyrics
    target_len = len(target_schema)
    del gen_schema[target_len:]
    del gen_lyrics[target_len:]
    
    # initialize array for the new fitted lyrics
    new_lyrics = []
    
    # loop through each line and either find existing line to place into the current position, 
    # or augment the current line. 
    
    # keep track of repeated lines
    repeats = np.array([])
    for num, line in enumerate(gen_lyrics):
        # if the line is already the right length, add it to the new lyrics. 
        if (gen_schema[num] == target_schema[num]): 
            new_lyrics.append(line)
            
        # if the line is not the right length, keep going until we find one that is
        elif (gen_schema[num] != target_schema[num]):
            # keep track of the number of syllables in the current line
            syls = gen_schema[num]
            temp_lyrics = gen_lyrics
            # keep track of which line we're on 
            track = 0
            # while the line is not the target length,
            while syls != target_schema[num]:
                
                if track == len(temp_lyrics)-1:
                    end = True
                    break
                else:
                    # go to the next line
                    track = track + 1
                    # check the num of syls
                    syls = gen_schema[track]
                    line = temp_lyrics[track]
                    end = False
                    
            # if we found a line that's the right length before we reached the end,
            # add the line to the new lyrics
            if end == False:
                new_lyrics.append(line)
                repeats = np.append(repeats, track)
                if np.count_nonzero(repeats == track) >= 3:
                    print("repeats greater than 3")
                    print(line)
                    temp_lyrics.pop(track)
                #temp_lyrics.pop(track)
                
            #If we didn't find a line that's the right length,
            if end:
                line = decontracted(line)
                print("No matching line")
                #print(line)
                while syls < target_schema[num]:
#                    print("not enough syls")
                    original_line = line
                    line = aug.augment(line)
                    line = re.sub(r'[^\w\s]','',line)
                    print(line)
                    syls = phoney.count_syllables(line)
                #In case we overshoot (add too many syllables)
                    if syls > target_schema[num]:
                        print("Oops we overshot")
                        line = original_line
                        syls = phoney.count_syllables(line)
                new_line = line
#                print("the same line:")
                print(line)
                syls = gen_schema[num]
                words = line.split(" ")
                while syls > target_schema[num]:
                    print("too many syls")
                    del words[-1]
                    new_line = ' '.join(words)
                    syls = phoney.count_syllables(new_line)
                    #In case too many syllables are deleted
                    if syls < target_schema[num]:
                        #print("Oops, deleted too many")
                        new_line = aug.augment(new_line)
                        syls = phoney.count_syllables(new_line)
                new_lyrics.append(new_line)
                
    return new_lyrics


## Time to test everything

In [68]:
artist = "Idina Menzel"
title = "Let it go"

In [130]:
target_lyrics = get_target_lyrics(artist, title)
gen_lyrics = lyrics_to_list('gpt2_gentext_slayer.txt')

Searching for "Idina Menzel" by Let it go...
Done.
Target lyrics:
['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, Heaven knows I tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on!', 'The cold never bothered me anyway', "It's funny how some distance", 'Makes everything seem small', 'And the fears that once controlled me', "Can't get to me at all", "It's time to see what I can do", 'To test the limits and break through', 'No right, no wrong, no rules for me', "I'm free", 'Let it go, let it go', "I'm one with the wind and sky", 'Let it go, let it

In [131]:
new_lyrics = fit_lyrics(gen_lyrics, target_lyrics)

['die by the sword', 'satan watches all of us', 'smiles as some do his bidding', 'try to escape the grasp of my hand', 'and your life will no longer exist', 'hear our cry, save us from', 'the hell in which we live', 'we turn our heads toward the sky', 'and listen for the steel', 'die by the sword', 'die by the sword', 'watch as flowers decay', 'on the cryptic life that died', 'the wisdom of the wizards', 'is only a nurtured lie', 'black knights of hells domain', 'walk upon the dead', 'satanas sits upon', 'the blood on which he feeds', 'die by the sword', 'die by the sword drones since the dawn of time', 'compelled to live your sheltered lives', 'not once has anyone ever seen', 'such a rise of pure hypocrisy', 'ill instigate ill free your mind', 'ill show you what ive known all this time', 'god hates us all, god hates us all', 'you know its true god hates this place', 'you know its true he hates this race', 'homicide-suicide', 'hate heals, you should try it sometime', 'strive for peace 

In [132]:
print(target_lyrics)

['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, Heaven knows I tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on!', 'The cold never bothered me anyway', "It's funny how some distance", 'Makes everything seem small', 'And the fears that once controlled me', "Can't get to me at all", "It's time to see what I can do", 'To test the limits and break through', 'No right, no wrong, no rules for me', "I'm free", 'Let it go, let it go', "I'm one with the wind and sky", 'Let it go, let it go', "You'll never see me cry", "Here I stand, and here I'll stay

In [133]:
print(gen_lyrics)

['die by the sword', 'smiles as some do his bidding', 'try to escape the grasp of my hand', 'and your life will no longer exist', 'hear our cry, save us from', 'the hell in which we live', 'and listen for the steel', 'die by the sword', 'die by the sword', 'watch as flowers decay', 'on the cryptic life that died', 'the wisdom of the wizards', 'is only a nurtured lie', 'black knights of hells domain', 'walk upon the dead', 'satanas sits upon', 'the blood on which he feeds', 'die by the sword', 'such a rise of pure hypocrisy', 'ill instigate ill free your mind', 'god hates us all, god hates us all', 'homicide-suicide', 'hate heals, you should try it sometime', 'strive for peace with acts of war', 'the']


In [134]:
print(new_lyrics)

['die by the sword drones since the dawn of time', 'satan watches all of us', 'compelled to live your sheltered lives', 'satan watches all of us', '1 vol ii vol ii 1 iii x vol iv the', 'die by the sword drones since the dawn of time', 'compelled to live your sheltered lives', 'die by the sword drones since the dawn of time', 'not once has anyone ever seen', 'die by the sword', 'the hell in which we live', 'satan watches all of us', 'the hell in which we live', 'the wisdom of the wizards', 'try to escape the grasp of my hand', 'walk upon the dead', 'compelled to live your sheltered lives', 'satan watches all of us', 'die by the sword', 'ill show you what ive known all this time', 'we turn our heads toward the sky', 'you know its true god hates this place', 'you know its true god hates this place', 'you know its true he hates this race', '']


In [ ]:
print(target_schema)
print(gen_schema)

In [ ]:
# End tensorflow session 
sess.close()